# Step 1: Evidence Details

|System|Source of data|Frameworks|Purpose|
|---|---|---|---|
|Intune|Intune Policy|Intune|Set administrative session timeout to 2 hours or less.|

```
RecomendedEvidenceName: AdministrativeSessionTimeoutReport
```



# Step 2: Define the System Specific Data


In [ ]:
# GET /identity/conditionalAccess/policies [use beta API]
{
    "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#identity/conditionalAccess/policies",
    "value": [
        {
            "id": "73e612e5-58c7-4901-be19-34d15de9b966",
            "templateId": null,
            "displayName": "TestUser MFA Policy",
            "createdDateTime": "2021-08-13T07:10:30.2425929Z",
            "modifiedDateTime": "2021-09-16T10:02:48.9276843Z",
            "state": "enabled",
            "sessionControls": null,
            "conditions": {
                "userRiskLevels": [],
                "signInRiskLevels": [],
                "clientAppTypes": [
                    "all"
                ],
                "servicePrincipalRiskLevels": [],
                "platforms": null,
                "locations": null,
                "devices": null,
                "clientApplications": null,
                "applications": {
                    "includeApplications": [
                        "797f4846-ba00-4fd7-ba43-dac1f8f63013"
                    ],
                    "excludeApplications": [],
                    "includeUserActions": [],
                    "includeAuthenticationContextClassReferences": [],
                    "applicationFilter": null
                },
                "users": {
                    "includeUsers": [
                        "7a669360-622d-4795-8a7e-b1c1fedf5ad7"
                    ],
                    "excludeUsers": [],
                    "includeGroups": [],
                    "excludeGroups": [],
                    "includeRoles": [],
                    "excludeRoles": [],
                    "includeGuestsOrExternalUsers": null,
                    "excludeGuestsOrExternalUsers": null
                }
            },
            "grantControls": {
                "operator": "OR",
                "builtInControls": [
                    "mfa"
                ],
                "customAuthenticationFactors": [],
                "termsOfUse": [],
                "authenticationStrength@odata.context": "https://graph.microsoft.com/v1.0/$metadata#identity/conditionalAccess/policies('73e612e5-58c7-4901-be19-34d15de9b966')/grantControls/authenticationStrength/$entity",
                "authenticationStrength": null
            }
        }
    ]
}

# GET /directoryRoles
{

    "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#directoryRoles",
    "value": [
        {
            "id": "0394c6e8-e8cc-4afa-85a3-8dc3a876fb65",
            "deletedDateTime": null,
            "description": "Can read everything that a Global Administrator can, but not update anything.",
            "displayName": "Global Reader",
            "roleTemplateId": "f2ef992c-3afb-46b9-b7cf-a126ee74c451"
        }
    ]
}

# GET /directoryRoles/{roleId}/members
{
    "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#directoryObjects",
    "value": [
        {
            "@odata.type": "#microsoft.graph.user",
            "id": "d618aa7d-eda0-4a13-88f4-67129cb7ba6d",
            "businessPhones": [],
            "displayName": "John Doe",
            "givenName": "John",
            "jobTitle": null,
            "mail": "john.doe@example.com",
            "mobilePhone": null,
            "officeLocation": null,
            "preferredLanguage": "en-US",
            "surname": "Doe",
            "userPrincipalName": "john.doe@example.com"
        },
    ]
}

# Step 2a: API & Flow


 - GET /users
  - The response will give a list of users.
  - For each role, use the below API to fetch the user's memberships:
 - GET /users/{userId}/memberOf
  - The response will give a list of memberships of the user, containing directory roles, groups, etc.
  - For each membership, we check if "@odata.type" field equals "#microsoft.graph.directoryRole".
  - If atleast one of the memberships satisfy the condition, that means the user is privileged.
  - Thus, we get a list of privileged users.
 - GET /identity/conditionalAccess/policies
  - The response will give the list of policies.
  - For each policy, check if sessionControl exists.
  - If it exists, we must ensure the below conditions are met:
    - state is 'enabled'.
    - sessionControls.signInFrequency is not null.
    - sessionControls.signInFrequency.isEnabled is true.
  - If sessionControls.signInFrequency.frequencyInterval is 'everyTime', then the policy is compliant.
  - Otherwise, we continue checking:
    - sessionControls.signInFrequency.type will be either hours or days.
    - sessionControls.signInFrequency.value will contain the number of hours or days.
    - We convert the value to minutes, then compare it with the rule input.
  - For each privileged user, check if atleast one compliant policy is attached.




# Step 2b: Define the Extended Schema


In [ ]:
# IntuneConditionalAccessPolicies
# Input:
# {
#     "AllowedSessionLifetimeMinutes": 120
# }
[
    {
        "PolicyID": "832c845a-fdd5-4fd9-93e0-ad8398f8d25a",
        "PolicyName": "Require Duo MFA",
        "PolicyStatus": "ACTIVE",
        "PolicyIncludedGroups": [],
        "PolicyExcludedGroups": [],
        "PolicyIncludedUsers": [
            {
                "UserId": "58dfe22d-043b-4154-a5dd-2896b3520309",
                "Name": "John Doe"
            }
        ],
        "PolicyExcludedUsers": [],
        "MaxSessionLifetimeMinutes": 120,
        "SigninFrequencyEnabled": True,
        "FrequencyIntervalIsEveryTime": False,
        "PolicyCreatedDate": "2023-06-30T15:46:12.5179773Z",
    }
]

# Step 3: Define the Standard Schema
    

In [ ]:
# AdministrativeSessionTimeoutReport
# Input:
# {
#     "AllowedSessionLifetimeMinutes": 120
# }
[
    {
        "System": "intune",
        "Source": "compliancecow",
        "ResourceID": "58dfe22d-043b-4154-a5dd-2896b3520309", # User ID
        "ResourceName": "John Doe", # User name
        "ResourceType": "N/A",
        "ResourceLocation": "N/A",
        "ResourceTags": "N/A",
        "UserEmail": "john.doe@example.com",
        "PolicyID": "832c845a-fdd5-4fd9-93e0-ad8398f8d25a",
        "PolicyName": "Require Duo MFA",
        "AllowedSessionLifetimeMinutes": 120, #User Input
        "MaxSessionLifetimeMinutes": 120,
        "SigninFrequencyEnabled": "true",
        "ValidationStatusCode": "TIMEOUT_LT_120_MIN",
        "ValidationStatusNotes": "The timeout for administrative sessions is set to less than 120 minutes",
        "ComplianceStatus": "COMPLIANT",
        "ComplianceStatusReason": "The timeout for administrative sessions is set to less than 120 minutes",
        "EvaluatedTime": "2024-04-23T07:54:43.012889Z",
        "UserAction": "",
        "ActionStatus": "",
        "ActionResponseURL": ""
    }
]

|System|Source|ResourceID|ResourceName|ResourceType|ResourceLocation|ResourceTags|UserEmail|PolicyID|PolicyName|AllowedSessionLifetimeMinutes|MaxSessionLifetimeMinutes|SigninFrequencyEnabled|ValidationStatusCode|ValidationStatusNotes|ComplianceStatus|ComplianceStatusReason|EvaluatedTime|UserAction|ActionStatus|ActionResponseURL|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|intune|compliancecow|58dfe22d-043b-4154-a5dd-2896b3520309|John Doe|N/A|N/A|N/A|john.doe@example.com|832c845a-fdd5-4fd9-93e0-ad8398f8d25a|Require Duo MFA|120|120|true|TIMEOUT_LT_120_MIN|The timeout for administrative sessions is set to less than 120 minutes|COMPLIANT|The timeout for administrative sessions is set to less than 120 minutes. Your proactive management of session timeouts ensures enhanced security and efficiency.|2024-04-23T07:54:43.012889Z||||

# Step 4: Describe the Compliance Taxonomy

|Validation Status Code|Validation Status Notes|Compliance Status|Compliance Status Reason|
|---|---|---|--|
|TIMEOUT_LT_{n}_MIN| The timeout for administrative sessions is set to less than {n} minutes|COMPLIANT| The timeout for administrative sessions is set to less than {n} minutes. Your proactive management of session timeouts ensures enhanced security and efficiency.|
|FREQ_INT_EVRY_TIME| The Session Frequency Interval is set to Every Time.|COMPLIANT| The Session Frequency Interval is set to Every Time. This option is evaluated on every sign-in attempt to an application in scope for this policy.|
|POLICY_NA| The user does not have any compliant policy attached.|NON_COMPLIANT| The user does not have any compliant policy attached. It is recommended to attach a policy with a session timeout set to less than {allowed_session_lifetime_minutes} minutes for improved security and operational efficiency.|

# Step 5: Calculation for Compliance Percentage and Status




In [ ]:
# Calculation of Compliance Percentage
CompliancePCT = (100 - (Count of 'NON_COMPLIANT' records * 100) / Total records)

# Compliance Status
COMPLIANT - 100%
NON_COMPLIANT - 0%>=status<100%
NOT_DETERMINED - If some error occurs.

# Step 6: Describe (in words) the Remediation Steps for Non-Compliance

FOR NOT_DETERMINED : UNKNOWN

FOR COMPLIANT : None

For NONCOMPLIANCE:

If Compliance Cow needs to notify the client, the following message can be sent via slack or ticket raised in JIRA:

Please ensure that the administrative session timeout is set to 2 hours or less.

# 7. Control Setup Details

| Control Details            |                                                 |
|----------------------------|-------------------------------------------------|
| **RuleName**               | EnsureAdministrativeSessionTimeoutIsLTTwoHrs    |
| **PreRequisiteRuleNames**  | GetIntuneUsers, IntuneConditionalAccessPolicies |
| **ExtendedSchemaRuleNames**| IntuneConditionalAccessPolicies                 |
| **ApplicationClassName**   | azureappconnector                               |
| **PostSynthesizerName**    | N/A                                             |